This script generates a basic connection to Carbon Intensity API
https://carbon-intensity.github.io/api-definitions/?python#carbon-intensity-api-v2-0-0

Then saves the extracted data into a csv file

In [1]:
#--------------------------------------------------------------------------
#Import required libraries
#--------------------------------------------------------------------------
import pandas as pd
import requests
from datetime import datetime, timedelta

In [2]:
#--------------------------------------------------------------------------
#API connection (test for only 1 day)
#--------------------------------------------------------------------------
headers = {
  'Accept': 'application/json'
}

base_url = 'https://api.carbonintensity.org.uk/intensity'
endpoint = 'stats'
from_date = '2024-01-01T00:00Z'
to_date = '2024-01-29T23:59Z'
from_date = datetime.fromisoformat(to_date[:-1]) - timedelta(days=30)
from_date = from_date.strftime('%Y-%m-%dT%H:%MZ')
block_date = 4
full_url = f"{base_url}/{endpoint}/{from_date}/{to_date}/{block_date}"
r = requests.get(full_url,
                 params={},
                 headers = headers)

In [3]:
#--------------------------------------------------------------------------
#Save data into variable
#--------------------------------------------------------------------------
#json_response = r.json()
#json_response.keys()

if r.status_code == 200:
    print("Success!")
    json_data = r.json()['data']
    df_carbon_intensity = pd.json_normalize(json_data)
else:
    print("Failure!")
    print(r.status_code)
    print(r.text)

Success!


In [5]:
print(json_data)

[{'from': '2023-12-31T23:59Z', 'to': '2024-01-01T03:59Z', 'intensity': {'max': 71, 'average': 65, 'min': 62, 'index': 'low'}}, {'from': '2024-01-01T03:59Z', 'to': '2024-01-01T07:59Z', 'intensity': {'max': 62, 'average': 59, 'min': 57, 'index': 'low'}}, {'from': '2024-01-01T07:59Z', 'to': '2024-01-01T11:59Z', 'intensity': {'max': 71, 'average': 54, 'min': 46, 'index': 'low'}}, {'from': '2024-01-01T11:59Z', 'to': '2024-01-01T15:59Z', 'intensity': {'max': 116, 'average': 105, 'min': 82, 'index': 'low'}}, {'from': '2024-01-01T15:59Z', 'to': '2024-01-01T19:59Z', 'intensity': {'max': 115, 'average': 100, 'min': 70, 'index': 'low'}}, {'from': '2024-01-01T19:59Z', 'to': '2024-01-01T23:59Z', 'intensity': {'max': 61, 'average': 48, 'min': 41, 'index': 'low'}}, {'from': '2024-01-01T23:59Z', 'to': '2024-01-02T03:59Z', 'intensity': {'max': 59, 'average': 55, 'min': 50, 'index': 'low'}}, {'from': '2024-01-02T03:59Z', 'to': '2024-01-02T07:59Z', 'intensity': {'max': 105, 'average': 75, 'min': 57, 'ind

In [6]:
#--------------------------------------------------------------------------
#Create table from last 6 months
#--------------------------------------------------------------------------
#Get current date
current_date = datetime.utcnow()

#Calculate dates
from_date = current_date - timedelta(days=180)
from_date_str = from_date.strftime('%Y-%m-%dT%H:%MZ')
to_date_str = current_date.strftime('%Y-%m-%dT%H:%MZ')

block_date = 4

#Empty df to save each iteration data
dfs = []

#Loop to get data in 30 days intervals (API limit)
while from_date < current_date:
    to_date = from_date + timedelta(days=30)
    to_date_str = to_date.strftime('%Y-%m-%dT%H:%MZ')
    
    full_url = f"{base_url}/{endpoint}/{from_date_str}/{to_date_str}/{block_date}"
    
    r = requests.get(full_url,
                     params={},
                     headers=headers)
    
    if r.status_code == 200:
        print(f"Success for {from_date_str} to {to_date_str}")
        json_data = r.json()['data']
        df_carbon_intensity = pd.json_normalize(json_data)
        dfs.append(df_carbon_intensity)
    else:
        print(r.status_code)
        print(r.text)

    #Next interval
    from_date = to_date + timedelta(days=1)
    from_date_str = from_date.strftime('%Y-%m-%dT%H:%MZ')

#Concatenate all df in one
df_carbon_intensity = pd.concat(dfs, ignore_index=True)

Success for 2023-08-07T16:37Z to 2023-09-06T16:37Z
Success for 2023-09-07T16:37Z to 2023-10-07T16:37Z
Success for 2023-10-08T16:37Z to 2023-11-07T16:37Z
Success for 2023-11-08T16:37Z to 2023-12-08T16:37Z
Success for 2023-12-09T16:37Z to 2024-01-08T16:37Z
Success for 2024-01-09T16:37Z to 2024-02-08T16:37Z


In [8]:
#DataFrame con los datos de carbon intensity de los ultimos 180 días
df_carbon_intensity.head()

,from,to,intensity.max,intensity.average,intensity.min,intensity.index
0,2023-08-07T16:37Z,2023-08-07T20:37Z,178,170,156,moderate
1,2023-08-07T20:37Z,2023-08-08T00:37Z,177,126,104,moderate
2,2023-08-08T00:37Z,2023-08-08T04:37Z,127,108,99,low
3,2023-08-08T04:37Z,2023-08-08T08:37Z,178,166,149,moderate
4,2023-08-08T08:37Z,2023-08-08T12:37Z,155,141,134,moderate


In [ ]:
#Guardar df a csv file
df_carbon_intensity.to_csv('../../data/raw/df_carbon_intensity.csv')